## 모듈 임포트

In [ ]:
import numpy as np
import pandas as pd

import tarfile
import json

import tensorflow as tf

## 데이터 가져오기

In [ ]:
lyric_filtered_path = '/content/mydrive/MyDrive/AIFFELTHON/mel_data/lyric_data.tsv'
mel_path = '/content/mydrive/MyDrive/AIFFELTHON/mel_data'

In [ ]:
metadata = pd.read_csv(lyric_filtered_path, sep='\t')
metadata.head()

,id,song_id,song_name,lyric,artist,genre,album,album_id,release
0,0,4639910,Feelings,Feelings nothing more than feelings Trying to ...,Various Artists,GN0900,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,20140512
1,8,1657318,Lovers’ Leap (Feat. Qypthone),i feel alright now but don't know how to speak...,애플스,GN0300,Sexualmetro,353020,20070625
2,9,30310140,"사랑, 그대라는 멜로디",그대 스치는 바람처럼 불어와서 내 곁에 머무른 사람 나도 몰래 내쉬는 숨처럼 익숙해...,진호,GN0100,Pastel Reflection,10047088,20170320
3,17,8068890,Girl Crush,이따 거기서 봐 이번엔 장담해 찾았어 Hot place 예감이 괜찮아 정말로 기대돼...,마마무 (Mamamoo),"GN2500,GN0200",Melting,2669407,20160226
4,19,5579388,무얼 기다리나 (Feat. 조원선),그냥 생각 없이 이렇다 할 뜻도 없이 쉼 없이 웃으며 떠드는 이들을 가만히 두리번거...,이영훈,"GN0500,GN0800",내가 부른 그림 2,2303168,20150205


In [ ]:
meta_id = metadata['song_id']
meta_id[:10]

0     4639910
1     1657318
2    30310140
3     8068890
4     5579388
5     5464935
6     3953129
7    31808054
8    30477250
9     3840610
Name: song_id, dtype: int64

In [ ]:
meta_id.isna().sum()

0

In [ ]:
meta_id.nunique()

214890

In [ ]:
len(meta_id)

214892

In [ ]:
'''
기본적으로 한 폴더에 최대 1천곡이 들어 있음 (e.g. folder27 -> 27000~27999)

아래 전처리는 CNN 구조에 사용할 (데이터 수, Hz, 길이(time), 채널(db)) shape로 바꿈
따라서 output tensor shape는 (폴더 내 데이터 수, 48, 1876, 1) 형태

<폴더 구조 예시>
song_id = 472831일 경우,
해당 파일은 mel_data/mel_4/arena_mel/72/472831

'''

def preprocessing(id_list, mel_path, dataset_size):
    
    f_list = []
    ex_list = []

    for song_id in id_list:
        # mel 파일이 존재하는 폴더 경로
        song_add = song_id // 1000

        # shape가 맞는 것만 가져오기
        mel_f = np.load(mel_path+f'/{song_add}/{song_id}.npy')
        
        if mel_f.shape == (48, 1876):
            f_list.append(mel_f)

        else:
            ex_list.append(song_id)
            print(f'FileShapeError: {song_id} file\'s shape doesn\'t match!')

    # stacking & reshape
    f_list = np.stack(f_list, axis=0)
    tensor_size = (dataset_size-len(ex_list), 48, 1876, 1)
    f_list = f_list.reshape(tensor_size)

    # add 100 (db processing)
    f_list += 100.

    return f_list, ex_list

print("ok!")

ok!


In [ ]:
for i in range(5):
    a = meta_i
    print(a)

20
20
20
20
20


In [ ]:
# id_list 정렬
meta_id = sorted(meta_id)

In [ ]:
meta_id[:5]

[2312, 2316, 2320, 2390, 2395]

In [ ]:
# test code
file_list, except_list = preprocessing(id_list=meta_id, mel_path=mel_path, dataset_size=1000)